In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # Librería para la visualización de datos
import seaborn as sns # Librería para la visualización de datos
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder # Librería para crear modelos de ML

In [2]:
np.random.seed(42)

n = 2000


In [3]:
# Fechas base
fechas_consulta = pd.date_range("2023-01-01", periods=n, freq="H")
fechas_inicio = fechas_consulta + pd.to_timedelta(np.random.randint(-300, 800, n), unit="m")
fechas_fin = fechas_inicio + pd.to_timedelta(np.random.randint(30, 300, n), unit="m")

C:\Users\lamas\AppData\Local\Temp\ipykernel_25200\1901837738.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas_consulta = pd.date_range("2023-01-01", periods=n, freq="H")


In [4]:
 #Dataset
df1 = pd.DataFrame({
    "ORIGEN": np.random.choice(["Madrid", "Barcelona", "Valencia", "Sevilla", "Bilbao"], n),
    "DESTINO": np.random.choice(["Madrid", "Barcelona", "Valencia", "Sevilla", "Bilbao"], n),
    "TIPO_TREN": np.random.choice(["AVE", "ALVIA", "MD", "R. EXPRESS", "OUIGO", "ESPECIAL_RARO"], n, p=[0.3,0.25,0.2,0.15,0.09,0.01]),
    "TIPO_TARIFA": np.random.choice(["Flexible", "Promo", "Mesa", "Especial", "UltraRara"], n),
    "CLASE": np.random.choice(["Turista", "Preferente", "Premium", "Rara"], n),
    "PRECIO": np.random.normal(50, 20, n),
    "FECHA_CONSULTA": fechas_consulta,
    "FECHA_INICIO": fechas_inicio,
    "FECHA_FIN": fechas_fin
})

In [5]:
# Introducimos nulos
for col in ["PRECIO", "TIPO_TARIFA", "CLASE"]:
    df1.loc[df1.sample(frac=0.05).index, col] = np.nan

In [6]:
# Introducimos duplicados
df1 = pd.concat([df1, df1.sample(50)], ignore_index=True)

In [7]:
# Outliers extremos
df1.loc[df1.sample(5).index, "PRECIO"] = [500, 600, 700, 800, 1000]

## Parte 1

In [8]:
# Muestra dimensiones (filas y columnas).
df1.shape

(2050, 9)

In [9]:
# Inspecciona las primeras y últimas 5 filas.
df1.head(5)


,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00
1,Bilbao,Barcelona,AVE,NaN,Preferente,47.013907,2023-01-01 01:00:00,2023-01-01 14:15:00,2023-01-01 19:02:00
2,Bilbao,Sevilla,AVE,Mesa,NaN,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00


In [10]:
df1.tail(5)

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN
2045,Bilbao,Sevilla,R. EXPRESS,Promo,Premium,29.027138,2023-01-12 07:00:00,2023-01-12 19:42:00,2023-01-12 22:20:00
2046,Bilbao,Bilbao,AVE,Especial,Preferente,21.140771,2023-03-11 18:00:00,2023-03-12 00:00:00,2023-03-12 02:52:00
2047,Bilbao,Sevilla,AVE,UltraRara,Turista,31.308899,2023-01-23 13:00:00,2023-01-23 22:12:00,2023-01-24 01:56:00
2048,Valencia,Madrid,AVE,Especial,Turista,79.153957,2023-01-13 18:00:00,2023-01-14 05:43:00,2023-01-14 07:12:00
2049,Madrid,Bilbao,AVE,Especial,Premium,49.911122,2023-01-24 00:00:00,2023-01-24 02:55:00,2023-01-24 06:11:00


In [11]:
# Lista los tipos de datos de cada columna.
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ORIGEN          2050 non-null   object        
 1   DESTINO         2050 non-null   object        
 2   TIPO_TREN       2050 non-null   object        
 3   TIPO_TARIFA     1948 non-null   object        
 4   CLASE           1950 non-null   object        
 5   PRECIO          1950 non-null   float64       
 6   FECHA_CONSULTA  2050 non-null   datetime64[ns]
 7   FECHA_INICIO    2050 non-null   datetime64[ns]
 8   FECHA_FIN       2050 non-null   datetime64[ns]
dtypes: datetime64[ns](3), float64(1), object(5)
memory usage: 144.3+ KB


In [12]:
#Número total de nulos por columna
df1.isnull().sum()

ORIGEN              0
DESTINO             0
TIPO_TREN           0
TIPO_TARIFA       102
CLASE             100
PRECIO            100
FECHA_CONSULTA      0
FECHA_INICIO        0
FECHA_FIN           0
dtype: int64

In [13]:
# Porcentaje de nulos sobre el total
(df1.isnull().sum()/len(df1))*100

ORIGEN            0.000000
DESTINO           0.000000
TIPO_TREN         0.000000
TIPO_TARIFA       4.975610
CLASE             4.878049
PRECIO            4.878049
FECHA_CONSULTA    0.000000
FECHA_INICIO      0.000000
FECHA_FIN         0.000000
dtype: float64

In [14]:
#Ordenar de mayor a menor
df1.isnull().sum().sort_values(ascending=False)

TIPO_TARIFA       102
PRECIO            100
CLASE             100
ORIGEN              0
TIPO_TREN           0
DESTINO             0
FECHA_CONSULTA      0
FECHA_INICIO        0
FECHA_FIN           0
dtype: int64

## Parte 2

In [15]:
# Visualiza las duplicadas.
df1[df1.duplicated()]

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN
2000,Madrid,Madrid,AVE,Mesa,Preferente,51.005582,2023-03-19 23:00:00,2023-03-20 05:11:00,2023-03-20 07:32:00
2001,Sevilla,Barcelona,AVE,Especial,Turista,34.576736,2023-01-14 12:00:00,2023-01-14 19:20:00,2023-01-14 22:08:00
2002,Sevilla,Valencia,ALVIA,Especial,Rara,58.227140,2023-02-01 07:00:00,2023-02-01 16:04:00,2023-02-01 17:47:00
2003,Sevilla,Valencia,ALVIA,Promo,Premium,12.260259,2023-01-30 09:00:00,2023-01-30 15:31:00,2023-01-30 20:17:00
2004,Barcelona,Madrid,OUIGO,Mesa,Rara,40.370870,2023-01-07 22:00:00,2023-01-07 20:06:00,2023-01-07 20:56:00
2005,Valencia,Bilbao,AVE,UltraRara,Rara,50.615720,2023-01-08 23:00:00,2023-01-09 01:32:00,2023-01-09 03:27:00
2006,Barcelona,Bilbao,MD,Flexible,Preferente,69.536715,2023-01-30 21:00:00,2023-01-30 20:02:00,2023-01-30 23:39:00
2007,Madrid,Bilbao,AVE,Flexible,Premium,39.494427,2023-01-07 11:00:00,2023-01-07 08:02:00,2023-01-07 11:33:00
2008,Barcelona,Valencia,ALVIA,UltraRara,Premium,68.661672,2023-03-05 09:00:00,2023-03-05 15:32:00,2023-03-05 18:13:00
2009,Sevilla,Madrid,MD,Mesa,Rara,37.354700,2023-01-31 08:00:00,2023-01-31 04:57:00,2023-01-31 07:50:00


In [16]:
#Elimina duplicados
df1.drop_duplicates().reset_index(drop=True)

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00
1,Bilbao,Barcelona,AVE,NaN,Preferente,47.013907,2023-01-01 01:00:00,2023-01-01 14:15:00,2023-01-01 19:02:00
2,Bilbao,Sevilla,AVE,Mesa,NaN,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00
...,...,...,...,...,...,...,...,...,...
1995,Valencia,Sevilla,AVE,NaN,Premium,43.653031,2023-03-25 03:00:00,2023-03-25 02:15:00,2023-03-25 06:04:00
1996,Sevilla,Valencia,ALVIA,Especial,Turista,39.725731,2023-03-25 04:00:00,2023-03-25 08:57:00,2023-03-25 12:49:00
1997,Barcelona,Bilbao,MD,UltraRara,Rara,55.218433,2023-03-25 05:00:00,2023-03-25 11:01:00,2023-03-25 12:31:00
1998,Valencia,Madrid,ALVIA,Promo,Premium,74.924525,2023-03-25 06:00:00,2023-03-25 04:54:00,2023-03-25 08:35:00


In [17]:
df1

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00
1,Bilbao,Barcelona,AVE,NaN,Preferente,47.013907,2023-01-01 01:00:00,2023-01-01 14:15:00,2023-01-01 19:02:00
2,Bilbao,Sevilla,AVE,Mesa,NaN,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00
...,...,...,...,...,...,...,...,...,...
2045,Bilbao,Sevilla,R. EXPRESS,Promo,Premium,29.027138,2023-01-12 07:00:00,2023-01-12 19:42:00,2023-01-12 22:20:00
2046,Bilbao,Bilbao,AVE,Especial,Preferente,21.140771,2023-03-11 18:00:00,2023-03-12 00:00:00,2023-03-12 02:52:00
2047,Bilbao,Sevilla,AVE,UltraRara,Turista,31.308899,2023-01-23 13:00:00,2023-01-23 22:12:00,2023-01-24 01:56:00
2048,Valencia,Madrid,AVE,Especial,Turista,79.153957,2023-01-13 18:00:00,2023-01-14 05:43:00,2023-01-14 07:12:00


## Parte 3

In [18]:
#Manipulacion de fechas, con este bucle podemos convertir cada una de la columnas en datetype con un solo código
for col in ["FECHA_CONSULTA", "FECHA_INICIO", "FECHA_FIN"]:
    df1[col] = pd.to_datetime(df1[col], errors= "coerce")

In [19]:
df1.dtypes

ORIGEN                    object
DESTINO                   object
TIPO_TREN                 object
TIPO_TARIFA               object
CLASE                     object
PRECIO                   float64
FECHA_CONSULTA    datetime64[ns]
FECHA_INICIO      datetime64[ns]
FECHA_FIN         datetime64[ns]
dtype: object

In [20]:
#Crear nuevas variables
df1["TIEMPO_VIAJE"] = (df1["FECHA_FIN"]- df1["FECHA_INICIO"])/ np.timedelta64(1,"m")
df1["INI_BUS"] = (df1["FECHA_INICIO"]- df1["FECHA_CONSULTA"])/ np.timedelta64(1,"m")

In [21]:
df1

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,298.0,560.0
1,Bilbao,Barcelona,AVE,NaN,Preferente,47.013907,2023-01-01 01:00:00,2023-01-01 14:15:00,2023-01-01 19:02:00,287.0,795.0
2,Bilbao,Sevilla,AVE,Mesa,NaN,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,58.0,744.0
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,242.0,-179.0
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,79.0,166.0
...,...,...,...,...,...,...,...,...,...,...,...
2045,Bilbao,Sevilla,R. EXPRESS,Promo,Premium,29.027138,2023-01-12 07:00:00,2023-01-12 19:42:00,2023-01-12 22:20:00,158.0,762.0
2046,Bilbao,Bilbao,AVE,Especial,Preferente,21.140771,2023-03-11 18:00:00,2023-03-12 00:00:00,2023-03-12 02:52:00,172.0,360.0
2047,Bilbao,Sevilla,AVE,UltraRara,Turista,31.308899,2023-01-23 13:00:00,2023-01-23 22:12:00,2023-01-24 01:56:00,224.0,552.0
2048,Valencia,Madrid,AVE,Especial,Turista,79.153957,2023-01-13 18:00:00,2023-01-14 05:43:00,2023-01-14 07:12:00,89.0,703.0


In [22]:
#Separar fecha en hora/día/mes
for col in ['FECHA_CONSULTA','FECHA_INICIO','FECHA_FIN']:
    df1[col + "_HORA"] = df1[col].dt.hour
    df1[col + "_NOMBREDIA"] = df1[col].dt.day_name()
    df1[col + "_DIA"] = df1[col].dt.day
    df1[col + "_MES"] = df1[col].dt.month


In [23]:
df1.head()

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,...,FECHA_CONSULTA_DIA,FECHA_CONSULTA_MES,FECHA_INICIO_HORA,FECHA_INICIO_NOMBREDIA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_NOMBREDIA,FECHA_FIN_DIA,FECHA_FIN_MES
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,298.0,...,1,1,9,Sunday,1,1,14,Sunday,1,1
1,Bilbao,Barcelona,AVE,NaN,Preferente,47.013907,2023-01-01 01:00:00,2023-01-01 14:15:00,2023-01-01 19:02:00,287.0,...,1,1,14,Sunday,1,1,19,Sunday,1,1
2,Bilbao,Sevilla,AVE,Mesa,NaN,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,58.0,...,1,1,14,Sunday,1,1,15,Sunday,1,1
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,242.0,...,1,1,0,Sunday,1,1,4,Sunday,1,1
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,79.0,...,1,1,6,Sunday,1,1,8,Sunday,1,1


## Parte 4

In [24]:
df1.describe()

,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_CONSULTA_HORA,FECHA_CONSULTA_DIA,FECHA_CONSULTA_MES,FECHA_INICIO_HORA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_DIA,FECHA_FIN_MES
count,1950.000000,2050,2050,2050,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000,2050.000000
mean,51.775879,2023-02-11 13:52:53.268292608,2023-02-11 18:12:04.360975616,2023-02-11 20:56:32.429268480,164.467805,259.184878,11.458049,14.571220,1.916098,11.487805,14.619024,1.920000,11.300488,14.622927,1.923902
min,-13.494979,2023-01-01 00:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,30.000000,-300.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,35.440948,2023-01-21 19:15:00,2023-01-22 01:20:00,2023-01-22 05:13:45,99.000000,-16.000000,5.000000,8.000000,1.000000,6.000000,8.000000,1.000000,5.000000,8.000000,1.000000
50%,50.751157,2023-02-11 14:30:00,2023-02-11 14:23:30,2023-02-11 15:58:30,166.000000,274.000000,11.000000,14.000000,2.000000,11.000000,14.500000,2.000000,11.000000,14.000000,2.000000
75%,64.130393,2023-03-04 08:45:00,2023-03-04 11:31:30,2023-03-04 14:32:00,230.000000,532.750000,17.000000,21.000000,3.000000,17.750000,21.000000,3.000000,17.000000,21.000000,3.000000
max,1000.000000,2023-03-25 07:00:00,2023-03-25 11:01:00,2023-03-25 12:49:00,299.000000,799.000000,23.000000,31.000000,3.000000,23.000000,31.000000,3.000000,23.000000,31.000000,3.000000
std,40.514955,NaN,NaN,NaN,76.047218,321.388604,6.939848,8.232294,0.810367,6.910648,8.207635,0.812566,6.945964,8.204009,0.811739


In [25]:
df1= df1[df1["PRECIO"]> 0]

In [26]:
df1.describe()

,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_CONSULTA_HORA,FECHA_CONSULTA_DIA,FECHA_CONSULTA_MES,FECHA_INICIO_HORA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_DIA,FECHA_FIN_MES
count,1941.000000,1941,1941,1941,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000
mean,52.044301,2023-02-11 09:52:12.612055552,2023-02-11 14:09:29.644513280,2023-02-11 16:54:22.009273600,164.872746,257.283874,11.440495,14.598145,1.909840,11.569809,14.634209,1.913962,11.350850,14.633179,1.918083
min,0.757598,2023-01-01 00:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,30.000000,-300.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,35.748427,2023-01-21 14:00:00,2023-01-21 15:11:00,2023-01-21 19:25:00,99.000000,-21.000000,5.000000,8.000000,1.000000,6.000000,8.000000,1.000000,5.000000,8.000000,1.000000
50%,50.962376,2023-02-11 04:00:00,2023-02-11 07:45:00,2023-02-11 10:33:00,166.000000,270.000000,11.000000,14.000000,2.000000,12.000000,15.000000,2.000000,11.000000,15.000000,2.000000
75%,64.258881,2023-03-04 06:00:00,2023-03-04 10:03:00,2023-03-04 12:53:00,230.000000,532.000000,17.000000,21.000000,3.000000,18.000000,21.000000,3.000000,17.000000,21.000000,3.000000
max,1000.000000,2023-03-25 07:00:00,2023-03-25 11:01:00,2023-03-25 12:49:00,299.000000,799.000000,23.000000,31.000000,3.000000,23.000000,31.000000,3.000000,23.000000,31.000000,3.000000
std,40.414863,NaN,NaN,NaN,76.140786,322.556130,6.917421,8.253539,0.813299,6.900612,8.230516,0.815643,6.969968,8.226147,0.814804


In [27]:
#Detección de outliers con el uso de quartiles IQR
q1 = df1["PRECIO"].quantile(0.25)
q3 = df1["PRECIO"].quantile(0.75)
q1

np.float64(35.74842674265345)

In [28]:
IQR = q3-q1

In [29]:
df1 = df1[(df1["PRECIO"] >= q1 - 1.5*IQR)& (df1["PRECIO"]<= q3 + 1.5*IQR)]

In [30]:
df1.describe()

,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_CONSULTA_HORA,FECHA_CONSULTA_DIA,FECHA_CONSULTA_MES,FECHA_INICIO_HORA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_DIA,FECHA_FIN_MES
count,1934.000000,1934,1934,1934,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000,1934.000000
mean,50.254603,2023-02-11 09:17:09.369182976,2023-02-11 13:34:15.170630656,2023-02-11 16:19:16.101344256,165.015512,257.096691,11.445191,14.596691,1.908997,11.570321,14.632368,1.913133,11.343330,14.631334,1.917270
min,0.757598,2023-01-01 00:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,30.000000,-300.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,35.708689,2023-01-21 14:00:00,2023-01-21 16:16:45,2023-01-21 20:18:00,99.000000,-21.000000,5.000000,8.000000,1.000000,6.000000,8.000000,1.000000,5.000000,8.000000,1.000000
50%,50.766680,2023-02-11 04:00:00,2023-02-11 07:36:30,2023-02-11 10:08:30,166.500000,269.500000,11.000000,14.000000,2.000000,12.000000,15.000000,2.000000,11.000000,14.500000,2.000000
75%,64.057389,2023-03-04 03:45:00,2023-03-04 09:02:45,2023-03-04 12:15:30,230.000000,530.750000,17.000000,21.000000,3.000000,18.000000,21.000000,3.000000,17.000000,21.000000,3.000000
max,106.546478,2023-03-25 07:00:00,2023-03-25 11:01:00,2023-03-25 12:49:00,299.000000,799.000000,23.000000,31.000000,3.000000,23.000000,31.000000,3.000000,23.000000,31.000000,3.000000
std,20.040325,NaN,NaN,NaN,76.140168,322.331564,6.914513,8.248953,0.812469,6.899018,8.226728,0.814828,6.969152,8.222342,0.813989


## Parte 5

In [31]:
# filtrar categorías poco usuales
def sacar_categorías_unusuales (data, columna, umbral):
    tabla = data[columna].value_counts()
    return tabla[tabla< umbral].index.tolist()



In [32]:
#valores extraños por columnas 
extraños_tarifa= sacar_categorías_unusuales(df1, "TIPO_TARIFA", 400)
extraños_tarifa

['Especial', 'UltraRara', 'Flexible', 'Promo', 'Mesa']

In [33]:
extraños_clase = sacar_categorías_unusuales(df1, "CLASE", 100)
extraños_clase

[]

In [34]:
df1 = df1[df1["TIPO_TARIFA"].isin(extraños_tarifa)]

In [35]:
df1

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,...,FECHA_CONSULTA_DIA,FECHA_CONSULTA_MES,FECHA_INICIO_HORA,FECHA_INICIO_NOMBREDIA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_NOMBREDIA,FECHA_FIN_DIA,FECHA_FIN_MES
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,298.0,...,1,1,9,Sunday,1,1,14,Sunday,1,1
2,Bilbao,Sevilla,AVE,Mesa,NaN,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,58.0,...,1,1,14,Sunday,1,1,15,Sunday,1,1
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,242.0,...,1,1,0,Sunday,1,1,4,Sunday,1,1
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,79.0,...,1,1,6,Sunday,1,1,8,Sunday,1,1
6,Bilbao,Sevilla,R. EXPRESS,Mesa,Rara,53.942367,2023-01-01 06:00:00,2023-01-01 02:27:00,2023-01-01 04:24:00,117.0,...,1,1,2,Sunday,1,1,4,Sunday,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,Bilbao,Sevilla,R. EXPRESS,Promo,Premium,29.027138,2023-01-12 07:00:00,2023-01-12 19:42:00,2023-01-12 22:20:00,158.0,...,12,1,19,Thursday,12,1,22,Thursday,12,1
2046,Bilbao,Bilbao,AVE,Especial,Preferente,21.140771,2023-03-11 18:00:00,2023-03-12 00:00:00,2023-03-12 02:52:00,172.0,...,11,3,0,Sunday,12,3,2,Sunday,12,3
2047,Bilbao,Sevilla,AVE,UltraRara,Turista,31.308899,2023-01-23 13:00:00,2023-01-23 22:12:00,2023-01-24 01:56:00,224.0,...,23,1,22,Monday,23,1,1,Tuesday,24,1
2048,Valencia,Madrid,AVE,Especial,Turista,79.153957,2023-01-13 18:00:00,2023-01-14 05:43:00,2023-01-14 07:12:00,89.0,...,13,1,5,Saturday,14,1,7,Saturday,14,1


In [36]:
#sustituir vaariables por valores calculadoso
df1["CLASE"] = df1["CLASE"].fillna(df1["CLASE"].mode()[0])
df1["PRECIO"] = df1["PRECIO"].fillna(df1["PRECIO"].mean())

In [37]:
df1.isnull().sum()

ORIGEN                      0
DESTINO                     0
TIPO_TREN                   0
TIPO_TARIFA                 0
CLASE                       0
PRECIO                      0
FECHA_CONSULTA              0
FECHA_INICIO                0
FECHA_FIN                   0
TIEMPO_VIAJE                0
INI_BUS                     0
FECHA_CONSULTA_HORA         0
FECHA_CONSULTA_NOMBREDIA    0
FECHA_CONSULTA_DIA          0
FECHA_CONSULTA_MES          0
FECHA_INICIO_HORA           0
FECHA_INICIO_NOMBREDIA      0
FECHA_INICIO_DIA            0
FECHA_INICIO_MES            0
FECHA_FIN_HORA              0
FECHA_FIN_NOMBREDIA         0
FECHA_FIN_DIA               0
FECHA_FIN_MES               0
dtype: int64

## Parte 6

In [38]:
# Calculo de nuevas variables que resulten interesantes para el análisis
df1["PRECIO_POR_MINUTO"] = df1["PRECIO"]/df1["TIEMPO_VIAJE"]

In [39]:
df1.head()

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,...,FECHA_CONSULTA_MES,FECHA_INICIO_HORA,FECHA_INICIO_NOMBREDIA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_NOMBREDIA,FECHA_FIN_DIA,FECHA_FIN_MES,PRECIO_POR_MINUTO
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,298.0,...,1,9,Sunday,1,1,14,Sunday,1,1,0.143230
2,Bilbao,Sevilla,AVE,Mesa,Rara,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,58.0,...,1,14,Sunday,1,1,15,Sunday,1,1,1.079057
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,242.0,...,1,0,Sunday,1,1,4,Sunday,1,1,0.233634
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,79.0,...,1,6,Sunday,1,1,8,Sunday,1,1,0.775514
6,Bilbao,Sevilla,R. EXPRESS,Mesa,Rara,53.942367,2023-01-01 06:00:00,2023-01-01 02:27:00,2023-01-01 04:24:00,117.0,...,1,2,Sunday,1,1,4,Sunday,1,1,0.461046


In [40]:
#Limpieza de columnas redundantes
cols_eliminar = [
    "FECHA_CONSULTA_HORA",
    "FECHA_CONSULTA_NOMBREDIA",
    "FECHA_CONSULTA_DIA",
    "FECHA_CONSULTA_MES"
]

df1.drop(columns=cols_eliminar, inplace=True)

In [41]:
df1.head()

,ORIGEN,DESTINO,TIPO_TREN,TIPO_TARIFA,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_INICIO_HORA,FECHA_INICIO_NOMBREDIA,FECHA_INICIO_DIA,FECHA_INICIO_MES,FECHA_FIN_HORA,FECHA_FIN_NOMBREDIA,FECHA_FIN_DIA,FECHA_FIN_MES,PRECIO_POR_MINUTO
0,Barcelona,Bilbao,R. EXPRESS,Especial,Turista,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,298.0,560.0,9,Sunday,1,1,14,Sunday,1,1,0.143230
2,Bilbao,Sevilla,AVE,Mesa,Rara,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,58.0,744.0,14,Sunday,1,1,15,Sunday,1,1,1.079057
3,Bilbao,Barcelona,ALVIA,Flexible,Turista,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,242.0,-179.0,0,Sunday,1,1,4,Sunday,1,1,0.233634
4,Valencia,Sevilla,OUIGO,Flexible,Preferente,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,79.0,166.0,6,Sunday,1,1,8,Sunday,1,1,0.775514
6,Bilbao,Sevilla,R. EXPRESS,Mesa,Rara,53.942367,2023-01-01 06:00:00,2023-01-01 02:27:00,2023-01-01 04:24:00,117.0,-213.0,2,Sunday,1,1,4,Sunday,1,1,0.461046


## Parte 7

In [42]:
#label encoder
le = LabelEncoder()
df1["CLASE"] = le.fit_transform(df1["CLASE"])

In [43]:
#OneHotEncoder
df1 = pd.get_dummies(df1, columns=["TIPO_TREN", "TIPO_TARIFA"], drop_first = True)

In [44]:
df1.head()

,ORIGEN,DESTINO,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_INICIO_HORA,...,PRECIO_POR_MINUTO,TIPO_TREN_AVE,TIPO_TREN_ESPECIAL_RARO,TIPO_TREN_MD,TIPO_TREN_OUIGO,TIPO_TREN_R. EXPRESS,TIPO_TARIFA_Flexible,TIPO_TARIFA_Mesa,TIPO_TARIFA_Promo,TIPO_TARIFA_UltraRara
0,Barcelona,Bilbao,3,42.682674,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,298.0,560.0,9,...,0.143230,False,False,False,False,True,False,False,False,False
2,Bilbao,Sevilla,2,62.585282,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,58.0,744.0,14,...,1.079057,True,False,False,False,False,False,True,False,False
3,Bilbao,Barcelona,3,56.539481,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,242.0,-179.0,0,...,0.233634,False,False,False,False,False,True,False,False,False
4,Valencia,Sevilla,0,61.265623,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,79.0,166.0,6,...,0.775514,False,False,False,True,False,True,False,False,False
6,Bilbao,Sevilla,2,53.942367,2023-01-01 06:00:00,2023-01-01 02:27:00,2023-01-01 04:24:00,117.0,-213.0,2,...,0.461046,False,False,False,False,True,False,True,False,False


In [45]:
#Escalado MinMaxScaler
scaler = MinMaxScaler()

In [46]:
df1[["PRECIO", "TIEMPO_VIAJE"]] = scaler.fit_transform(df1[["PRECIO", "TIEMPO_VIAJE"]])

In [47]:
df1.head()

,ORIGEN,DESTINO,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_INICIO_HORA,...,PRECIO_POR_MINUTO,TIPO_TREN_AVE,TIPO_TREN_ESPECIAL_RARO,TIPO_TREN_MD,TIPO_TREN_OUIGO,TIPO_TREN_R. EXPRESS,TIPO_TARIFA_Flexible,TIPO_TARIFA_Mesa,TIPO_TARIFA_Promo,TIPO_TARIFA_UltraRara
0,Barcelona,Bilbao,3,0.396309,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,0.996283,560.0,9,...,0.143230,False,False,False,False,True,False,False,False,False
2,Bilbao,Sevilla,2,0.584444,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,0.104089,744.0,14,...,1.079057,True,False,False,False,False,False,True,False,False
3,Bilbao,Barcelona,3,0.527294,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,0.788104,-179.0,0,...,0.233634,False,False,False,False,False,True,False,False,False
4,Valencia,Sevilla,0,0.571970,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,0.182156,166.0,6,...,0.775514,False,False,False,True,False,True,False,False,False
6,Bilbao,Sevilla,2,0.502744,2023-01-01 06:00:00,2023-01-01 02:27:00,2023-01-01 04:24:00,0.323420,-213.0,2,...,0.461046,False,False,False,False,True,False,True,False,False


In [48]:
# Valores inconsistentes
df1[df1["FECHA_FIN"] < df1["FECHA_INICIO"]]

,ORIGEN,DESTINO,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_INICIO_HORA,...,PRECIO_POR_MINUTO,TIPO_TREN_AVE,TIPO_TREN_ESPECIAL_RARO,TIPO_TREN_MD,TIPO_TREN_OUIGO,TIPO_TREN_R. EXPRESS,TIPO_TARIFA_Flexible,TIPO_TARIFA_Mesa,TIPO_TARIFA_Promo,TIPO_TARIFA_UltraRara


In [49]:
df1 = df1[df1["FECHA_FIN"] >= df1["FECHA_INICIO"]]

In [50]:
df1.head()

,ORIGEN,DESTINO,CLASE,PRECIO,FECHA_CONSULTA,FECHA_INICIO,FECHA_FIN,TIEMPO_VIAJE,INI_BUS,FECHA_INICIO_HORA,...,PRECIO_POR_MINUTO,TIPO_TREN_AVE,TIPO_TREN_ESPECIAL_RARO,TIPO_TREN_MD,TIPO_TREN_OUIGO,TIPO_TREN_R. EXPRESS,TIPO_TARIFA_Flexible,TIPO_TARIFA_Mesa,TIPO_TARIFA_Promo,TIPO_TARIFA_UltraRara
0,Barcelona,Bilbao,3,0.396309,2023-01-01 00:00:00,2023-01-01 09:20:00,2023-01-01 14:18:00,0.996283,560.0,9,...,0.143230,False,False,False,False,True,False,False,False,False
2,Bilbao,Sevilla,2,0.584444,2023-01-01 02:00:00,2023-01-01 14:24:00,2023-01-01 15:22:00,0.104089,744.0,14,...,1.079057,True,False,False,False,False,False,True,False,False
3,Bilbao,Barcelona,3,0.527294,2023-01-01 03:00:00,2023-01-01 00:01:00,2023-01-01 04:03:00,0.788104,-179.0,0,...,0.233634,False,False,False,False,False,True,False,False,False
4,Valencia,Sevilla,0,0.571970,2023-01-01 04:00:00,2023-01-01 06:46:00,2023-01-01 08:05:00,0.182156,166.0,6,...,0.775514,False,False,False,True,False,True,False,False,False
6,Bilbao,Sevilla,2,0.502744,2023-01-01 06:00:00,2023-01-01 02:27:00,2023-01-01 04:24:00,0.323420,-213.0,2,...,0.461046,False,False,False,False,True,False,True,False,False
